# Enable GPU

In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# Actor Critic Share Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ActorCritic_Net(nn.Module):
  def __init__(self, input_dims, output_dims, fc1_dims = 128):
    super(ActorCritic_Net , self).__init__()
    self.fc1 = nn.Linear(input_dims,fc1_dims)
    self.actor = nn.Linear(fc1_dims, output_dims)
    self.critic = nn.Linear(fc1_dims,1)

  def forward(self, state):
    x = F.relu(self.fc1(state))
    pi = F.softmax(self.actor(x), dim = 1)
    value = self.critic(x)
    return (pi, value)



# REINFORCE with Baseline Agent

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical 
import numpy as np

class ActorCritic(nn.Module):
  def __init__(self, input_dims, output_dims, fc1_dims = 128, gamma = 0.99, lr = 1e-3):
    super(ActorCritic, self).__init__()
    self.ac_net = ActorCritic_Net(input_dims= input_dims, output_dims= output_dims, fc1_dims = fc1_dims)
    self.optimizer = optim.RMSprop(params= self.ac_net.parameters(), lr = lr)
    self.gamma = gamma

    self.log_probs = []
    self.values = []
  
  def get_action(self, state):
    pi, v = self.ac_net(state)
  
    distribution = Categorical(probs = pi)
    action = distribution.sample()
    self.log_probs.append(distribution.log_prob(action))
    self.values.append(v)

    return action


  def learn_mean(self, rewards ,states, actions, return_norm = True):
    
  
    returns = []
    actor_losses = []
    critic_losses = []

    # Calculate returns
    G = 0
    for reward in rewards[::-1]:
      G = reward + self.gamma * G
      returns.insert(0, G)
    returns = torch.tensor(returns).to(device)


    if return_norm:
      eps = np.finfo(np.float32).eps.item()
      returns = (returns - returns.mean()) / (returns.std() + eps)
    
    # Resize the vectors
  
    self.values = torch.cat(self.values).squeeze() # concatinating plus squeeze since input dim is 2d
    self.log_probs = torch.cat(self.log_probs) # only concatinating since input dim is 1d

    # Compute actor and critic losses
    
    for G, log_prob, v in zip(returns, self.log_probs, self.values):
      
      G = G.detach() 
      advantage = G - v.item() # detach the grad computation to avoid computing gradient
      actor_losses.append(-log_prob * advantage)
      critic_losses.append(F.smooth_l1_loss(v, G))
    self.optimizer.zero_grad()
    loss = (torch.stack(actor_losses).sum()).mean() + (torch.stack(critic_losses).sum()).mean()
    loss.backward()
    self.optimizer.step()

    # clear out the memory
    self.values = []
    self.log_probs = []

  def learn_forward(self, rewards, states, actions, return_norm = True):
    
  
    returns = []
    states = torch.cat(states, dim = 0).to(device)
    actions = torch.tensor(actions).to(device)  
    rewards = torch.tensor(rewards).to(device).flip(dims= [0])

    # Calculate returns
    G = 0
    for reward in rewards:
      G = reward + self.gamma * G
      returns.insert(0, G)
    returns = torch.tensor(returns).to(device)

    if return_norm:
      eps = np.finfo(np.float32).eps.item()
      returns = (returns - returns.mean()) / (returns.std() + eps)
    

    # Compute actor and critic losses

    for G, state, action in zip(returns, states, actions):
      G = G.detach() 
      state = state.unsqueeze(0)
      pi, v = self.ac_net(state)
      dist = Categorical(probs = pi)
      log_prob = dist.log_prob(action)
      advantage = G - v.item() # detach the grad computation to avoid computing gradient
      actor_loss = -log_prob * advantage
      critic_loss = F.smooth_l1_loss(v, torch.tensor([G]).unsqueeze(0).to(device))
      self.optimizer.zero_grad()
      loss = actor_loss + critic_loss
      loss.backward()
      self.optimizer.step()

    # clear out the memory
    self.values = []
    self.log_probs = []

  def learn_backward(self, rewards, states, actions, return_norm = True):
    
    returns = []
    states = torch.cat(states, dim = 0).to(device).flip(dims = [0])
    actions = torch.tensor(actions).to(device).flip(dims = [0])
    rewards = torch.tensor(rewards).to(device).flip(dims= [0])

    # Calculate returns
    G = 0
    for reward in rewards:
      G = reward + self.gamma * G
      returns.insert(0, G)
    returns = torch.tensor(returns).to(device)
    returns = returns.flip(dims = [0]) # Flip it again to traverse backward

    if return_norm:
      eps = np.finfo(np.float32).eps.item()
      returns = (returns - returns.mean()) / (returns.std() + eps)
    

    # Compute actor and critic losses

    for G, state, action in zip(returns, states, actions):
      G = G.detach() 
      state = state.unsqueeze(0)
      pi, v = self.ac_net(state)
      dist = Categorical(probs = pi)
      log_prob = dist.log_prob(action)
      advantage = G - v.item() # detach the grad computation to avoid computing gradient
      actor_loss = -log_prob * advantage
      critic_loss = F.smooth_l1_loss(v, torch.tensor([G]).unsqueeze(0).to(device)).unsqueeze(0)
      self.optimizer.zero_grad()
      assert actor_loss.size() == critic_loss.size()
      loss = actor_loss + critic_loss
      loss.backward()
      self.optimizer.step()

    # clear out the memory
    self.values = []
    self.log_probs = []


# Without Wandb

In [6]:
import gym
import torch
import time

def train():

  start = time.time()

  env = gym.make('CartPole-v1')
  env.seed(543)
  torch.manual_seed(543)

  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.n

  device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
  agent = ActorCritic(input_dims = state_dim,  output_dims = action_dim, lr = 0.001).to(device)
  num_ep = 3000
  print_interval = 100
  running_score = 10

  for ep in range(num_ep):
    state = env.reset()
    score = 0
    done = False
    rewards = []
    states = []
    actions = []

    while not done:
      state = torch.tensor([state]).float().to(device)
      action = agent.get_action(state)
      next_state, reward, done, _ = env.step(action.item())
      
      # saving episode
      rewards.append(reward)
      states.append(state)
      actions.append(action.item())
      # update score and state
      score += reward
      state = next_state

      if done:
        break

  
    # calculating score and running score
    running_score = 0.05 * score + (1 - 0.05) * running_score

    # train the agent
    agent.learn_backward(rewards, states, actions, return_norm = True)

    if ep % print_interval == 0:
      print('episode {} average reward {}, ended at {:.01f}'.format(ep, running_score, time.time() - start))
  

In [ ]:
train()

# With Wandb

In [ ]:
!pip install wandb
!wandb login

In [ ]:
import wandb
sweep_config = dict()
sweep_config['method'] = 'grid'
sweep_config['metric'] = {'name': 'running_score', 'goal': 'maximize'}
sweep_config['parameters'] = {'learning': {'values': ['learn_mean','learn_forward','learn_backward']}, 'learning_rate': {'values' : [0.01, 0.001, 0.0001,0.0003,0.00001]}
                              , 'norm_return': {'value': True}, 'num_neurons': {'value': 128}}

sweep_id = wandb.sweep(sweep_config, project = 'REINFORCE_Baseline')

Create sweep with ID: uticyas7
Sweep URL: https://wandb.ai/ko120/REINFORCE_Baseline/sweeps/uticyas7


In [ ]:
import gym 
import torch
import time
import wandb

def train():
  wandb.init(config = {'env':'CartPole-v1','algorithm:': 'REINFORCE_Baseline','architecture': 'shared','num_laeyrs':'2'}, project = 'REINFORCE_Baseline',group = 'Cart_REINFORCE_Baseline_with_128_neurons_RMSProp')
  config = wandb.config

  start = time.time()

  env = gym.make('CartPole-v1')
  env.seed(543)
  torch.manual_seed(543)

  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.n

  device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
  agent = ActorCritic(input_dims = state_dim,  output_dims = action_dim, lr = config.learning_rate, fc1_dims= config.num_neurons).to(device)
  num_ep = 3000
  print_interval = 100
  save_interval = 1000
  running_score = 10

  wandb.watch(agent)
  for ep in range(1,num_ep+1):
    state = env.reset()
    score = 0
    done = False
    rewards = []
    states = []
    actions = []
    while not done:
      state = torch.tensor([state]).float().to(device)
      action = agent.get_action(state)
      next_state, reward, done, _ = env.step(action.item())
      
      # saving episode
      rewards.append(reward)
      states.append(state)
      actions.append(action.item())

      # update score and state
      score += reward
      state = next_state

      if done:
        break
  
    # calculating score and running score
    running_score = 0.05 * score + (1 - 0.05) * running_score
    wandb.log({'episode': ep, 'running_score': running_score})

    # train the agent
  
    if config.learning == 'learn_mean':
      agent.learn_mean(rewards,states, actions, return_norm = True)
    elif config.learning == 'learn_forward':
      agent.learn_forward(rewards, states, actions, return_norm = True)
    elif config.learning == 'learn_backward':
      agent.learn_backward(rewards, states, actions, return_norm = True)

    if ep % print_interval == 0:
      print('episode {} average reward {}, ended at {:.01f}'.format(ep, running_score, time.time() - start))    
    
    if ep % save_interval == 0:
      save_name = 'agent_' + str(ep) + '.pt'
      torch.save(agent.state_dict(),save_name)
      wandb.save(save_name)

    if ep == num_ep:
      dummy_input = torch.rand(1,4).to(device)
      torch.onnx.export(agent.ac_net,dummy_input,'final_model.onnx')
      wandb.save('final_model.onnx')
    

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: sa12jwhb with config:
wandb: 	learning: learn_mean
wandb: 	learning_rate: 0.01
wandb: 	norm_return: True
wandb: 	num_neurons: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep


episode 100 average reward 59.97391039064079, ended at 7.0
episode 200 average reward 61.89096405838821, ended at 22.4
episode 300 average reward 287.4959541386293, ended at 67.4
episode 400 average reward 441.40501878301546, ended at 126.3
episode 500 average reward 386.12215299398105, ended at 191.9
episode 600 average reward 497.46071711913794, ended at 285.3
episode 700 average reward 469.79866650347964, ended at 366.6
episode 800 average reward 471.2086979303794, ended at 445.6
episode 900 average reward 499.82954025480495, ended at 542.2
episode 1000 average reward 439.45723819070673, ended at 630.4
episode 1100 average reward 499.64155480962796, ended at 728.4
episode 1200 average reward 499.99787781477613, ended at 825.4
episode 1300 average reward 497.060488231461, ended at 920.0
episode 1400 average reward 478.8244108332537, ended at 1013.6
episode 1500 average reward 461.9305351492036, ended at 1108.4
episode 1600 average reward 446.1696586564902, ended at 1178.4
episode 170

episode,3000
running_score,499.92257
_runtime,2279
_timestamp,1627082513
_step,3000


episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
running_score,▁▂▂▃▃▆▄▇██▆██▇██████▅▇▇▇██▆███▆▇▇▄▆▅▅▇▇█
_runtime,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Agent Starting Run: 8ef9mlqk with config:
wandb: 	learning: learn_mean
wandb: 	learning_rate: 0.001
wandb: 	norm_return: True
wandb: 	num_neurons: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

wandb: Agent Starting Run: 7urrzckl with config:
wandb: 	learning: learn_backward
wandb: 	learning_rate: 1e-05
wandb: 	norm_return: True
wandb: 	num_neurons: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep


episode 100 average reward 22.755056258088082, ended at 9.8
episode 200 average reward 22.290075668268017, ended at 20.0
episode 300 average reward 18.065510604768143, ended at 29.8
episode 400 average reward 23.844039612709008, ended at 41.1
episode 500 average reward 27.368024955917125, ended at 53.9
episode 600 average reward 29.092309375654686, ended at 66.6
episode 700 average reward 24.627655672379724, ended at 79.3
episode 800 average reward 25.093283083045897, ended at 91.5
episode 900 average reward 27.507672337342314, ended at 104.5
episode 1000 average reward 27.85420258490636, ended at 117.2
episode 1100 average reward 28.44699977824292, ended at 131.2
episode 1200 average reward 25.674642547149496, ended at 143.6
episode 1300 average reward 39.10251351900882, ended at 159.6
episode 1400 average reward 28.777853219849504, ended at 174.1
episode 1500 average reward 32.71761911911992, ended at 189.7
episode 1600 average reward 34.41307239995592, ended at 205.9
episode 1700 av

episode,3000
running_score,68.951
_runtime,522
_timestamp,1627115289
_step,2999


episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
running_score,▁▁▁▁▂▁▂▂▂▂▂▂▂▂▃▂▂▃▃▂▃▃▃▄▃▅▄▃▄▄▅▅▅▆▄▅█▇▇█
_runtime,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇██
_timestamp,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# You can see the result here!
[Report Link](https://wandb.ai/ko120/REINFORCE_Baseline/reports/REINFORCE-with-Baseline-forward-and-backward--Vmlldzo4NzM4ODE)